In [1]:
from pathlib import Path
import json

from inference_parquet import process_embeddings
from train_linear_model_slim import train_and_save
from active_learning_from_parquet import plot_logits

2024-01-16 01:10:53.900207: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-16 01:10:53.902301: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-16 01:10:53.944374: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-16 01:10:53.946322: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-16 01:10:54.943547: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [4]:
unlabelled_source = "/mnt/input"
labelled_source = "/mnt/labelled"
model_output_file = "/mnt/output/pw_model.keras"
embeddings_dir = "/mnt/embeddings/"
search_results_dir = "/mnt/output/search_results"

skip_if_file_exists = True
embedding_model_version = 4

print(Path(unlabelled_source).exists())
print(Path(labelled_source).exists())
print(Path(model_output_file).parent.exists())
print(Path(embeddings_dir).exists())
print(Path(search_results_dir).parent.exists())




True
True
True
True
True


In [5]:


if skip_if_file_exists and Path(model_output_file).exists() and Path(model_output_file + '.labels.json').exists():
    classifier_model = model_output_file
    with open(model_output_file + '.labels.json', 'r') as file:
        labels = json.load(file)
else:
    classifier_model, labels = train_and_save(labelled_source, model_output_file, embedding_model_version)

In [6]:
process_embeddings(embeddings_dir, (classifier_model, labels), search_results_dir, skip_if_file_exists)

processing 17923 embeddings files


Processing: 100%|██████████| 17923/17923 [1:55:14<00:00,  2.59it/s]  

finished processing 17923 embeddings files


In [ ]:
plot_logits(inference_results, target_class = "pos", target_logit = None)

In [ ]:
display_search_results(inference_results, labels, target_label='pos')